## Running the models

For each of the deformation types—viscous, elastic, and plastic, we will run the corresponding ASPECT parameter file.

In [ ]:
%%capture
! aspect ../prm_files/elastic.prm

In [ ]:
%%capture
! aspect ../prm_files/viscous.prm

In [ ]:
%%capture
! aspect ../prm_files/plastic.prm

> In this above cell, it is assumed that the ASPECT directory is installed system-wide or that you are using the HUBzero ASPECT installation. If not, modify the ASPECT executable to the location where it is installed.

In [ ]:
# Load the relevant libraries
import numpy as np
import pyvista as pv
import glob
from IPython.display import HTML

### Plotting the solution field

ASPECT outputs the solution fields in binary, .vtu, format. We will use the python package `pyvista` to read the output files and `matplotlib` to plot the solution fields.

>The mesh in all three cases is structured, we don't have to worry about interpolating the solution to a structured grid for plotting.

In [ ]:
def plot_deformation(output_dir, field, label):
    '''
    Process a series of VTU files in the given output directory, returning the 'field'
    data for plotting.
    Inputs:
    -----------
    output_dir : str
        Path to the folder containing solution VTU files (expected in solution/solution-*.vtu)
    field : str
        Name of the field we want to look at in the solution
    label : str
        Label for the field being plotted
    '''

    solution_file_names = sorted(glob.glob(output_dir + 'solution/solution-*.pvtu'))

    plotter = pv.Plotter()
    plotter = pv.Plotter(off_screen=False)
    mesh = pv.read(solution_file_names[0])

    # Create the first frame
    plotter.add_mesh(mesh, scalars=field, show_scalar_bar=False)
    plotter.add_axes()

    # We know that the model is XY plane so the following
    # lines just sets up the camera
    plotter.view_xy()
    plotter.camera.roll = 0
    plotter.camera.parallel_projection = True

    gif_file = output_dir.split('/')[0] + '.gif'
    plotter.open_gif(gif_file, fps=4)

    # Update function for each frame
    def update(frame):
        frame_int = int(frame)
        new_mesh = pv.read(solution_file_names[frame_int])
        plotter.add_mesh(new_mesh, scalars=field, cmap='RdBu', \
                         scalar_bar_args={"vertical": True, "title": "Slab"})
        plotter.add_text(f"Time step: {frame}", name="tstep")
        plotter.add_text(output_dir.split('/')[0],  position='upper_right',
                        font_size=14)

    n_frames = len(solution_file_names)

    for i in range(n_frames):
        update(i)
        plotter.write_frame()

    plotter.close()

In [ ]:
plot_deformation('elastic_deformation/', 'slab', 'Slab Deformation')
plot_deformation('viscous_deformation/', 'slab', 'Slab Deformation')
plot_deformation('plastic_deformation/', 'slab', 'Slab Deformation')

In [ ]:
# Now, you would have the gifs saved in the current directory
# this cell visualizes the model outputs side by side for
# comparison

HTML(f"""
<div style="display: flex; justify-content: center; align-items: center; gap: 10px;">
    <img src="{'elastic_deformation.gif'}" width="500">
    <img src="{'viscous_deformation.gif'}" width="500">
    <img src="{'plastic_deformation.gif'}" width="500">
</div>
""")

Notice, the different deformation styles of the slab under similar external forces.

 &nbsp;<div style="text-align: right">  
    &rarr; <b>NEXT: [Stokes flow benchmark](./7_stokes_flow_benchmark.ipynb) </b> <a href=""></a> &nbsp;&nbsp;
     <img src="../assets/education-gem-notebooks_icon.png" alt="icon"  style="width:4%">
  </div>